## Electric Vehicle Population Data - 2025 
This dataset from the Washington State Open Data Portal contains information about Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through the Washington State Department of Licensing (DOL). Each record represents an individual vehicle and reflects the growing population of electric vehicles in Washington. 

Access & Use Information
* Source: https://data.wa.gov/Transportation/Electric-Vehicle-Population-Data/f6w7-q2d2/about_data
* Public: This dataset is intended for public access and use.
* License: Open Data Commons Open Database License (ODbL) v1.0

The goal of this notebook is to explore the Washington State EV dataset using Vega-Altair, a Python library for creating interactive visualizations: https://altair-viz.github.io/

### 1. Import data and packages

In [ ]:
#!pip install --upgrade altair
#!pip install altair vega_datasets
#!pip install vegafusion

In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data

alt.data_transformers.disable_max_rows()
alt.data_transformers.enable("vegafusion")

In [ ]:
df = pd.read_csv("Electric_Vehicle_Population_Data_20250825.csv")

### 2. Explore the dataset

In [ ]:
print("Number of records: ", len(df))

In [ ]:
df.head()

### 3. Define Questions

**Question 1: What is the most common EV type?**

In [ ]:
# Summarize data so it renders quicker
summary = df['Electric Vehicle Type'].value_counts().to_frame(name='Count').reset_index()

In [ ]:
# Color palette to use throughout 
categorical = 'set2'
sequential = 'blues'

In [ ]:
# Simple bar chart showing number of BEVs and PHEVs in WA
alt.Chart(summary).mark_bar().encode(
    y = 'Electric Vehicle Type',
    x = 'Count', 
    color = alt.Color('Electric Vehicle Type', scale = alt.Scale(range = ['#a6d854', '#8da0cb'])), 
    tooltip = ['Count']
)

In [ ]:
# Now lets view the same data, but this time as a stacked bar chart showing the overall percentange of BEVs vs PHEVs
alt.Chart(summary).transform_joinaggregate(
    TotalEVs='sum(Count)',
).transform_calculate(
    PercentOfWAEVs="datum.Count / datum.TotalEVs"
).mark_bar().encode(
    alt.X('PercentOfWAEVs:Q').axis(format='.0%', title = 'WA State EV Types'), 
    color = alt.Color('Electric Vehicle Type', scale = alt.Scale(range = ['#a6d854', '#8da0cb'])), 
    tooltip = ['Electric Vehicle Type', alt.Tooltip('PercentOfWAEVs:Q', format='.1%', title='Percentage')]
)

As you can see, Battery Electric Vehicles are far more common than Plug-in Hybrid Electric Vehicles, accounting for about 80% of the Electric Vehicles in Washington State

## **Question 2: What are the most common makes and models of EVs?**

In [ ]:
## Summarize EVS by make
df['Make'].value_counts()

As you can see, there are many EV makes. Let's look at the top 5 manufacturers

In [ ]:
# summarize top 5 makes
summary = df['Make'].value_counts().to_frame(name='Count').reset_index()
summary.head(5).style.hide(axis="index")

In [ ]:
# Now lets visualize the top models for Tesla, Chevrolet, Nissan, Ford, and Kia in a side by side bar chart

# create a df with just Tesla, Chevy, Nissan, Ford, and Kia
top_5_makes = ['TESLA', 'CHEVROLET', 'NISSAN', 'FORD', 'KIA']
df_top5 = df[df['Make'].isin(top_5_makes)]

selection = alt.selection_point(fields=["Make"])
base =  alt.Chart(df_top5).properties(width=250, height=250)

# color scheme, set2. drop blue and green that have already been used. 
overview_colors = ["#66c2a5", "#fc8d62", "#e78ac3", "#ffd92f", "#e5c494", "#b3b3b3"]

# overview chart showing top 5 makes
overview = alt.Chart(df_top5).mark_bar().encode(
    y = alt.Y('count()').title("Count"),
    x = alt.X("Make").sort('-y'),
    color = alt.condition(selection, alt.value('#e78ac3'), alt.value("lightgrey")), 
    tooltip = [
        alt.Tooltip('Make'),
        alt.Tooltip('count()', title = 'Count')]
).add_params(selection).properties(height=250, width=250, title = 'Top EV Makes')

# detail chart showing models within each make
detail = base.mark_bar().encode(
    y = alt.Y("count()").title("Count"),
    x = alt.X("Model").sort('-y'),
    color = alt.Color('Electric Vehicle Type', scale = alt.Scale(range = ['#a6d854', '#8da0cb'])), 
    tooltip = [
        alt.Tooltip('Model'),
        alt.Tooltip('count()', title = 'Count')]
).transform_filter(selection).properties(height=250, width=250, title = 'Top EV Models')

filtered_chart = overview | detail

In [ ]:
filtered_chart

## **Question 3: Which Washington state counties have the most EVs?**

In [ ]:
# summarize EV count by county
# vehicles registered in WA may reside outside of WA, but for the purposes of this analysis, I only want to look at vehicles located in WA
county_summary = df[df['State'] == 'WA']
county_summary = county_summary['County'].value_counts().to_frame(name='Count').reset_index()

As you can see, most EVs are registered in King, Snohomish, and Pierce counties. Let's visualize the number of EVs in each county on a map of Washington state. 

In [ ]:
# First we need to pull in a map of the US counties from the vega datasets
# Citation: I used ChatGPT to learn out to how to pull in a map of all US counties
counties = alt.topo_feature(data.us_10m.url, 'counties')

In [ ]:
# The TopoJSON maps counties by FIPS code, whereas the EV dataset refers to county by name. 
# I found a list of all the WA state counties and their FIPS code here: https://unicede.air-worldwide.com/unicede/unicede_washington_fips.html
# Then I created a csv file with the state code, county code, combined code, and county name
FIPS_codes = pd.read_csv('WA_FIPS.csv')
FIPS_codes.head()

In [ ]:
# Now we can pull in the FIPS Code and add it to the EV dataset
county_summary['FIPS Code'] = county_summary['County'].map(FIPS_codes.set_index("County Name")["FIPS Code"])

In [ ]:
# Now we create a choropleth map with the number of EVs by county
# Citation: I used ChatGPT to learn out to how to filter for WA state and create the choropleth map
# The Altair documentation is also useful: https://altair-viz.github.io/altair-tutorial/notebooks/09-Geographic-plots.html

alt.Chart(counties).mark_geoshape(
    stroke="white"
).transform_filter(
    "floor(datum.id / 1000) == 53"
).transform_lookup(
    lookup="id",   
    from_=alt.LookupData(county_summary, "FIPS Code", ["County", "Count"])
).encode(
    color=alt.Color("Count:Q", scale=alt.Scale(scheme = sequential)),
    tooltip=["County:N", "Count:Q"]
).project("mercator").properties(
    width=400,
    height=400,
    title="Electric Vehicles by County in Washington"
)

As you can see, the majority of EVs are located in King county, followed up by Snohomish and Pierce counties. This probably makes sense given that EV chargers tend to be located in more populated areas. 